<a href="https://colab.research.google.com/github/yujin-jo/mini-project-team1/blob/main/analysis_04_%EB%B0%B0%EA%B8%89%EC%82%AC%EB%B9%84%EC%A4%91%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 세팅
- 깃허브 클론 
- 드라이브 마운트 
- 파일 임포트

In [ ]:
!git clone https://github.com/yujin-jo/mini-project-team1.git

Cloning into 'mini-project-team1'...
remote: Enumerating objects: 588, done.
remote: Counting objects: 100% (588/588), done.
remote: Compressing objects: 100% (552/552), done.
remote: Total 588 (delta 167), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (588/588), 807.79 KiB | 5.65 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as   np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

In [21]:
movies = pd.read_csv("/content/mini-project-team1/1. Crawling/data/KOBIS_TOT (1).csv")
movies["개봉일"] = pd.to_datetime(movies["개봉일"])
movies.sample(10)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
502,503,2013,슈퍼배드 2,"크리스 리노드,피에르 꼬팽",유니버설픽쳐스인터내셔널 코리아(유),2013-09-12,미국,421,6359208000,925108,애니메이션,전체관람가,일반영화
1721,1722,2016,부릉! 부릉! 브루미즈: 스피더의 모험 일기,이영준,(주)삼지애니메이션,2016-12-15,한국,194,81128500,10208,애니메이션,전체관람가,독립/예술영화
1313,1314,2015,어떤살인,안용훈,"TCO(주)더콘텐츠온,(주)전망좋은 영화사",2015-10-28,한국,179,137025800,17190,스릴러,청소년관람불가,독립/예술영화
2481,2482,2018,킨: 더 비기닝,"조나단 베이커,조쉬 베이커",판씨네마(주),2018-10-11,미국,167,151798300,17543,미스터리,15세이상관람가,일반영화
1253,1254,2015,미안해 사랑해 고마워,전윤수,(주)쇼박스,2015-10-28,한국,344,359247800,51207,드라마,15세이상관람가,일반영화
1826,1827,2017,분노의 질주: 더 익스트림,F. 게리 그레이,유니버설픽쳐스인터내셔널 코리아(유),2017-04-12,미국,1469,29981156348,3653238,액션,15세이상관람가,일반영화
3144,3145,2020,비밀정보원: 인 더 프리즌,안드레아 디 스테파노,(주)제이앤씨미디어그룹,2020-04-15,영국,200,172588940,20317,범죄,청소년관람불가,일반영화
2863,2864,2019,경계선,알리 아바시,(주)제이앤씨미디어그룹,2019-10-24,스웨덴,48,38676620,17462,판타지,청소년관람불가,독립/예술영화
2280,2281,2018,궁합,홍창표,(주)씨제이이엔엠,2018-02-28,한국,966,10478875438,1340117,코미디,12세이상관람가,일반영화
2815,2816,2019,주전장,미키 데자키,(주)시네마달,2019-07-25,미국,56,35376040,39980,다큐멘터리,전체관람가,독립/예술영화


---
# I. 가설 설정과 분석 배경

1. 가설  :  2011년 ~ 2020년까지 국내 영화 배급사에서 CJ E&M의 비중은 연도 별로 감소했는가? 
2. 가설을 선택한 이유 : 
    - 기존에는 국내 영화의 경우 CJ E&M이 독점했왔음. 
    - 다른 신생 배급사들은 없었을까 궁금했음 
    -  기존에 있던 영세 배급사들은 성장하거나 망했을까 궁금했고, 여전히 CJ E&M이 영화 산업을 지배하고 있는지 확인하고 싶었음 

# II. 분석 방향 
1. 활용 데이터 
  - 영화 개봉연도 : 연도 별로 영화 수 집계 필요 
  - 영화 배급사 : 영화 별 배급사 정보 (KOBIS 참고) 
  - 영화 수 : 연도 별, 배급사 별로 집계한 영화 수

2. 분석 방법론 
  - 시계열 분석 활용 (2011 ~ 2020년 영화 정보) 
  - 영화 배급사의 종류가 연도 별로 어떻게 달라졌는지 확인  (중소 배급사 등 배급사가 늘었는지 확인하기 위함)
  - 개봉 연도와 제작사 별로 영화 편수를 카운트한다. 
  - 연도 별로 제작사 별 비중을 계산하여 추이를 확인한다. 

3. 시각화 
  - 비율 바 차트




# III. 분석 과정  

## 1. 데이터 테이블 분석
- 2011년 ~ 2020년까지의 총 3,375개의 영화 데이터가 들어감 
- movie 테이블에 들어간 정보를 movies.info()로 파악 
- 영화 배급사의 종류는 '씨제이이엔엠', '롯데쇼핑(주)롯데엔터테인먼트', '유니버설픽쳐스인터네셔널 코리아(유)', '(주)넥스트엔터테인먼트월드 (NEW)', '(주) 팝엔터테인먼트',

In [132]:
# 영화 기본 정보 확인 
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      3375 non-null   int64         
 1   개봉연도    3375 non-null   int64         
 2   영화명     3375 non-null   object        
 3   감독      3375 non-null   object        
 4   배급사     3374 non-null   object        
 5   개봉일     3375 non-null   datetime64[ns]
 6   국적      3375 non-null   object        
 7   전국스크린수  3375 non-null   int64         
 8   전국매출액   3375 non-null   int64         
 9   전국관객수   3375 non-null   int64         
 10  장르      3375 non-null   object        
 11  등급      3375 non-null   object        
 12  영화구분    3375 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 342.9+ KB


,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
2381,2382,2018,출국,노규엽,NaN,2018-11-14,한국,274,639521850,81722,드라마,15세이상관람가,독립/예술영화


In [133]:
movies[movies['배급사'].isnull()]

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
2381,2382,2018,출국,노규엽,NaN,2018-11-14,한국,274,639521850,81722,드라마,15세이상관람가,독립/예술영화


In [35]:
# 영화 배급사 종류 확인: 우리나라 내 
a = movies[movies['배급사'].str.contains("씨제이이엔엠") == True]
a

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
0,1,2011,트랜스포머 3,마이클 베이,(주)씨제이이엔엠,2011-06-29,미국,1409,74841350500,7784807,액션,12세이상관람가,일반영화
2,3,2011,써니,강형철,(주)씨제이이엔엠,2011-05-04,한국,472,53773170600,7328820,드라마,15세이상관람가,일반영화
3,4,2011,쿵푸팬더 2,여인영,(주)씨제이이엔엠,2011-05-26,미국,948,44298724500,5062722,애니메이션,전체관람가,일반영화
4,5,2011,완득이,이한,(주)씨제이이엔엠,2011-10-20,한국,500,38526161500,5309928,드라마,12세이상관람가,일반영화
5,6,2011,미션임파서블:고스트프로토콜,브래드 버드,(주)씨제이이엔엠,2011-12-15,미국,1038,38258970000,5042164,액션,15세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,3112,2020,아이즈 온 미 : 더 무비,박재석,씨제이포디플렉스 주식회사,2020-06-10,한국,124,506274000,39228,공연,전체관람가,일반영화
3112,3113,2020,다만 악에서 구하소서 파이널컷,홍원찬,(주)씨제이이엔엠,2020-10-28,한국,475,326106570,37961,액션,청소년관람불가,일반영화
3161,3162,2020,극장판 요괴워치: 포에버 프렌즈,타카하시 시게하루,(주)씨제이이엔엠,2020-10-15,일본,422,120684750,15386,애니메이션,전체관람가,일반영화
3166,3167,2020,엔딩스 비기닝스,드레이크 도리머스,"(주)스마일이엔티,(주)씨제이이엔엠",2020-06-24,미국,251,110180880,13401,드라마,15세이상관람가,독립/예술영화


In [54]:
new_movies = movies['배급사'].str.split(", ").tolist()
new_movies.sample(150)
# new_df = pd.DataFrame(df.City.str.split('|').tolist(), index=df.EmployeeId).stack()
# new_df = new_df.reset_index([0, 'EmployeeId'])
# new_df.columns = ['EmployeeId', 'City']

AttributeError: ignored

In [141]:
new_movies = movies.assign(배급사=movies.배급사.str.split(","))
new_movies = new_movies.배급사.apply(pd.Series).merge(new_movies, right_index = True, left_index = True)
new_movies.sample(10)
# df_melt = df.assign(names=df.names.str.split(","))
# print(df_melt)

,0,1,2,3,4,5,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
1802,시네마 뉴원,(주)드림팩트엔터테인먼트,NaN,NaN,NaN,NaN,1803,2016,스퀴즈,테리 재스트로,"[시네마 뉴원, (주)드림팩트엔터테인먼트]",2016-11-10,미국,40,14809000,1866,코미디,15세이상관람가,일반영화
137,유니버설픽쳐스인터내셔널 코리아(유),NaN,NaN,NaN,NaN,NaN,138,2011,타워 하이스트,브렛 래트너,[유니버설픽쳐스인터내셔널 코리아(유)],2011-11-17,미국,213,631715500,90840,액션,12세이상관람가,일반영화
1630,(주)박수엔터테인먼트,NaN,NaN,NaN,NaN,NaN,1631,2016,동급생,나카무라 쇼코,[(주)박수엔터테인먼트],2016-05-19,일본,100,280704500,38706,애니메이션,15세이상관람가,독립/예술영화
175,워너브러더스 코리아(주),NaN,NaN,NaN,NaN,NaN,176,2011,히어애프터,클린트 이스트우드,[워너브러더스 코리아(주)],2011-03-24,미국,112,336316000,45755,드라마,12세이상관람가,일반영화
370,(주)씨제이이엔엠,NaN,NaN,NaN,NaN,NaN,371,2012,레드라이트,로드리고 코르테스,[(주)씨제이이엔엠],2012-08-23,스페인,197,355743500,49925,미스터리,15세이상관람가,일반영화
1656,(주)박수엔터테인먼트,NaN,NaN,NaN,NaN,NaN,1657,2016,키즈모노가타리 I : 철혈편,신보 아키유키,[(주)박수엔터테인먼트],2016-06-30,일본,74,187057500,24492,애니메이션,15세이상관람가,일반영화
1831,(주)쇼박스,NaN,NaN,NaN,NaN,NaN,1832,2017,프리즌,나현,[(주)쇼박스],2017-03-23,한국,1047,23790276987,2931897,범죄,청소년관람불가,일반영화
1128,워너브러더스 코리아(주),NaN,NaN,NaN,NaN,NaN,1129,2015,하트 오브 더 씨,론 하워드,[워너브러더스 코리아(주)],2015-12-03,미국,535,6776484023,809327,액션,12세이상관람가,일반영화
3077,롯데컬처웍스(주)롯데엔터테인먼트,NaN,NaN,NaN,NaN,NaN,3078,2020,수퍼 소닉,제프 파울러,[롯데컬처웍스(주)롯데엔터테인먼트],2020-02-12,미국,609,932457150,118725,애니메이션,전체관람가,일반영화
157,유니버설픽쳐스인터내셔널 코리아(유),NaN,NaN,NaN,NaN,NaN,158,2011,황당한 외계인: 폴,그렉 모톨라,[유니버설픽쳐스인터내셔널 코리아(유)],2011-04-07,미국,148,461937500,61808,코미디,15세이상관람가,일반영화


In [88]:
new_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   0       3374 non-null   object        
 1   1       321 non-null    object        
 2   2       15 non-null     object        
 3   3       1 non-null      object        
 4   4       1 non-null      object        
 5   5       1 non-null      object        
 6   순번      3375 non-null   int64         
 7   개봉연도    3375 non-null   int64         
 8   영화명     3375 non-null   object        
 9   감독      3375 non-null   object        
 10  배급사     3374 non-null   object        
 11  개봉일     3375 non-null   datetime64[ns]
 12  국적      3375 non-null   object        
 13  전국스크린수  3375 non-null   int64         
 14  전국매출액   3375 non-null   int64         
 15  전국관객수   3375 non-null   int64         
 16  장르      3375 non-null   object        
 17  등급      3375 non-null   object        
 18  영화구분    

In [124]:
new_movies = movies.assign(배급사=movies.배급사.str.split(","))
new_movies = new_movies.배급사.apply(pd.Series).merge(new_movies, right_index = True, left_index = True)
new_new_movies = new_movies.melt(id_vars = ["순번", "개봉연도", "영화명", "감독", "배급사", "개봉일", "국적", "전국스크린수", "전국매출액", "전국관객수", "장르", "등급", "영화구분"], var_name="new_배급사", value_name = "진짜배급사")
new_movies_a = new_new_movies.sort_values(by = ["순번", "new_배급사"], ascending = True)
new_movies = new_movies_a[new_movies_a["진짜배급사"].notnull()]

0           [(주)씨제이이엔엠]
1       [롯데쇼핑㈜롯데엔터테인먼트]
2           [(주)씨제이이엔엠]
3           [(주)씨제이이엔엠]
4           [(주)씨제이이엔엠]
             ...       
3370         [노바엔터테인먼트]
3371        [(주)영화사 그램]
3372      [(주)더블앤조이픽쳐스]
3373         [(주)엣나인필름]
3374     [(주)제이앤씨미디어그룹]
Name: 배급사, Length: 3375, dtype: object

In [130]:
new_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3713 entries, 0 to 3374
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   순번       3713 non-null   int64         
 1   개봉연도     3713 non-null   int64         
 2   영화명      3713 non-null   object        
 3   감독       3713 non-null   object        
 4   배급사      3713 non-null   object        
 5   개봉일      3713 non-null   datetime64[ns]
 6   국적       3713 non-null   object        
 7   전국스크린수   3713 non-null   int64         
 8   전국매출액    3713 non-null   int64         
 9   전국관객수    3713 non-null   int64         
 10  장르       3713 non-null   object        
 11  등급       3713 non-null   object        
 12  영화구분     3713 non-null   object        
 13  new_배급사  3713 non-null   object        
 14  진짜배급사    3713 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(9)
memory usage: 464.1+ KB


In [109]:
new_movies_a = new_new_movies.sort_values(by = ["순번", "new_배급사"], ascending = True)

In [114]:
new_movies = new_movies_a[new_movies_a["진짜배급사"].notnull()].head(60)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분,new_배급사,진짜배급사
0,1,2011,트랜스포머 3,마이클 베이,[(주)씨제이이엔엠],2011-06-29,미국,1409,74841350500,7784807,액션,12세이상관람가,일반영화,0,(주)씨제이이엔엠
1,2,2011,최종병기 활,김한민,[롯데쇼핑㈜롯데엔터테인먼트],2011-08-10,한국,615,55808856500,7466976,사극,15세이상관람가,일반영화,0,롯데쇼핑㈜롯데엔터테인먼트
2,3,2011,써니,강형철,[(주)씨제이이엔엠],2011-05-04,한국,472,53773170600,7328820,드라마,15세이상관람가,일반영화,0,(주)씨제이이엔엠
3,4,2011,쿵푸팬더 2,여인영,[(주)씨제이이엔엠],2011-05-26,미국,948,44298724500,5062722,애니메이션,전체관람가,일반영화,0,(주)씨제이이엔엠
4,5,2011,완득이,이한,[(주)씨제이이엔엠],2011-10-20,한국,500,38526161500,5309928,드라마,12세이상관람가,일반영화,0,(주)씨제이이엔엠
5,6,2011,미션임파서블:고스트프로토콜,브래드 버드,[(주)씨제이이엔엠],2011-12-15,미국,1038,38258970000,5042164,액션,15세이상관람가,일반영화,0,(주)씨제이이엔엠
6,7,2011,조선명탐정 : 각시투구꽃의 비밀,김석윤,[(주)쇼박스],2011-01-27,한국,611,35841769500,4786259,코미디,12세이상관람가,일반영화,0,(주)쇼박스
7,8,2011,도가니,황동혁,[(주)씨제이이엔엠],2011-09-22,한국,644,35566854800,4662822,드라마,청소년관람불가,일반영화,0,(주)씨제이이엔엠
8,9,2011,해리포터와 죽음의 성물2,데이빗 예이츠,[워너브러더스 코리아(주)],2011-07-13,미국,757,34507819000,4400298,판타지,전체관람가,일반영화,0,워너브러더스 코리아(주)
9,10,2011,캐리비안의 해적: 낯선 조류,롭 마샬,[한국소니픽쳐스릴리징브에나비스타영화㈜],2011-05-19,미국,960,27279967500,3130046,액션,12세이상관람가,일반영화,0,한국소니픽쳐스릴리징브에나비스타영화㈜


In [137]:
a_movies = movies["배급사"].str.split(","))
a_movies = a_movies.배급사.apply(pd.Series).merge(a_movies, right_index = True, left_index = True)
a_movies.sample(10)
# df_melt = df.assign(names=df.names.str.split(","))
# print(df_melt)

AttributeError: ignored